### Imports

In [1]:
import numpy as np
import os,cv2,math,glob,random
import scipy.io as sio
from math import cos, sin
from pathlib import Path
import pandas as pd
import mediapipe
import warnings
import matplotlib.pyplot as plt
import math
warnings.filterwarnings('ignore')


objc[14397]: Class CaptureDelegate is implemented in both /Users/omarmarie/miniforge3/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x131ea2538) and /Users/omarmarie/miniforge3/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x140648860). One of the two will be used. Which one is undefined.
objc[14397]: Class CVWindow is implemented in both /Users/omarmarie/miniforge3/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x131ea2588) and /Users/omarmarie/miniforge3/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x13730ca68). One of the two will be used. Which one is undefined.
objc[14397]: Class CVView is implemented in both /Users/omarmarie/miniforge3/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x131ea25b0) and /Users/omarmarie/miniforge3/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x13730ca90). One of the two will be used. Which one is undefined.
objc[14397]: Class CVSlider is implemented in both /U

### Read Data

In [2]:
data = pd.read_csv('prepared_data.csv', index_col = False)

In [3]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,929,930,931,932,933,934,935,yaw,pitch,roll
0,218.50034,309.08737,220.16293,287.94670,220.39182,291.27646,215.88454,253.16727,220.78503,279.62137,...,221.71300,243.67117,225.13382,299.59800,212.06892,304.29495,208.75371,1.044306,-22.874239,4.908886
1,198.65656,288.92084,187.82204,266.46198,201.99747,274.69626,196.23444,245.24129,187.14044,259.58466,...,230.24248,225.32686,231.67642,263.04114,225.04077,267.36460,222.86319,68.155240,26.932743,17.243670
2,143.05011,359.64900,144.70465,349.49045,143.29271,351.96910,140.05186,335.75400,144.76701,345.59012,...,324.39163,147.87735,325.51035,167.89949,320.82858,169.79830,319.33835,50.485413,-10.579652,-13.570645
3,226.72110,312.40903,211.26804,294.25903,220.25150,296.42953,199.99522,265.63297,207.77931,286.92654,...,229.78252,220.76442,233.40524,268.02110,205.04506,271.19662,200.10342,17.143373,-10.048455,-21.392782
4,229.61539,301.07340,221.73444,272.16202,224.89871,281.02000,210.32718,244.07130,219.69594,263.11774,...,219.90984,227.54619,221.72044,271.09393,203.71938,275.06375,201.86015,0.685565,-1.536199,-12.643008


In [4]:
features = data.iloc[:, 0   : 936]
labels   = data.iloc[:, 936 :    ]

### Separete fetures data into x and y coordinates

In [5]:
y_cord_features = features[[i for i in features if int(i)%2 != 0]]
x_cord_features = features[[i for i in features if int(i)%2 == 0]]

In [6]:
x_cord_features.head()

,0,2,4,6,8,10,12,14,16,18,...,916,918,920,922,924,926,928,930,932,934
0,218.50034,220.16293,220.39182,215.88454,220.78503,222.16890,226.45067,163.52289,228.89633,229.77367,...,229.25603,234.22491,243.80159,227.42400,226.67778,251.91934,246.82832,243.67117,299.59800,304.29495
1,198.65656,187.82204,201.99747,196.23444,187.14044,192.09712,208.92330,208.96887,213.38713,212.46323,...,198.98555,201.01733,216.93471,196.61096,200.45555,235.13998,230.15826,225.32686,263.04114,267.36460
2,143.05011,144.70465,143.29271,140.05186,144.76701,144.25558,142.39409,111.22898,141.61957,141.54236,...,147.98055,149.95276,152.69232,147.47470,146.70361,150.43501,148.62198,147.87735,167.89949,169.79830
3,226.72110,211.26804,220.25150,199.99522,207.77931,205.69737,203.12099,162.70569,196.96803,192.65569,...,222.93045,225.97426,239.92307,221.04677,222.63947,228.40356,223.97792,220.76442,268.02110,271.19662
4,229.61539,221.73444,224.89871,210.32718,219.69594,217.62376,213.42474,161.11057,210.89825,208.80525,...,230.21333,233.68443,243.02412,229.06137,228.90870,234.21666,229.82881,227.54619,271.09393,275.06375


In [7]:
y_cord_features.head()

,1,3,5,7,9,11,13,15,17,19,...,917,919,921,923,925,927,929,931,933,935
0,309.08737,287.94670,291.27646,253.16727,279.62137,265.92420,228.16650,202.98181,201.66109,189.21115,...,288.53403,285.99026,286.50043,289.97260,290.03162,218.24280,221.71300,225.13382,212.06892,208.75371
1,288.92084,266.46198,274.69626,245.24129,259.58466,251.39012,230.73274,225.73300,214.54941,206.09431,...,269.87190,267.96848,274.27847,270.10710,272.13998,228.09627,230.24248,231.67642,225.04077,222.86319
2,359.64900,349.49045,351.96910,335.75400,345.59012,339.97385,326.04016,323.38046,316.91104,311.96338,...,349.95990,348.62946,348.96548,350.53043,350.92460,323.00490,324.39163,325.51035,320.82858,319.33835
3,312.40903,294.25903,296.42953,265.63297,286.92654,274.63065,241.31706,243.58446,217.08853,206.60855,...,291.56354,287.90878,284.60022,293.36108,293.53427,225.44104,229.78252,233.40524,205.04506,200.10342
4,301.07340,272.16202,281.02000,244.07130,263.11774,251.18056,222.61673,231.16109,205.93282,195.07910,...,273.07568,269.47394,273.19380,274.23740,276.15125,216.87988,219.90984,221.72044,203.71938,201.86015


In [8]:
def data_preprocess(features):
    
    y_cord_features = features[[i for i in features if int(i)%2 != 0]]
    x_cord_features = features[[i for i in features if int(i)%2 == 0]]
    
    point_to_center_x = 2
    point_to_center_y = 3


    centered_x_features = x_cord_features.sub(x_cord_features.iloc\
                                          [:, int(point_to_center_x)], axis='rows')

    centered_y_features = y_cord_features.sub(y_cord_features.iloc\
                                          [:, int(point_to_center_y)], axis='rows')
    
    
    
    
    
    # Divide distances 1
    centered_x_features = centered_x_features.sub(centered_x_features.min(axis=1),\
                                                  axis=0)
    
    centered_x_features = centered_x_features.div( (centered_x_features.max(axis=1)\
                                                    -centered_x_features.min(axis=1))\
                                                  ,axis=0)

    centered_y_features = centered_y_features.sub(centered_y_features.min(axis=1),axis=0)
    
    centered_y_features = centered_y_features.div( (centered_y_features.max(axis=1)\
                                                    -centered_y_features.min(axis=1)) ,axis=0)
    
    
    # Divide distances 2
#     x1, y1 = x_cord_features.iloc[:, 150], y_cord_features.iloc[:, 150]
#     x2, y2 = x_cord_features.iloc[:, 100], y_cord_features.iloc[:, 100]
    
#     distances = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    
#     scaled_x_features = centered_x_features.divide(distances, axis='rows')
#     scaled_y_features = centered_y_features.divide(distances, axis='rows')
    
    scaled_data = pd.concat([centered_x_features, centered_y_features], axis=1)
    
    return scaled_data
    

In [9]:
scaled_data = data_preprocess(features)

In [10]:
scaled_data.head()

,0,2,4,6,8,10,12,14,16,18,...,917,919,921,923,925,927,929,931,933,935
0,0.427961,0.435677,0.436740,0.415821,0.438565,0.444987,0.464860,0.172802,0.476210,0.480282,...,0.654941,0.644471,0.646571,0.660862,0.661105,0.365628,0.379911,0.393991,0.340217,0.326572
1,0.086924,0.015460,0.108961,0.070948,0.010965,0.043659,0.154643,0.154944,0.184086,0.177992,...,0.572409,0.560926,0.598992,0.573828,0.586091,0.320394,0.333341,0.341992,0.301962,0.288825
2,0.590029,0.608007,0.592665,0.557452,0.608684,0.603127,0.582902,0.244280,0.574486,0.573647,...,0.576204,0.563124,0.566428,0.581813,0.585688,0.311203,0.324836,0.335835,0.289807,0.275156
3,0.448939,0.371288,0.416430,0.314643,0.353757,0.343296,0.330349,0.127264,0.299431,0.277762,...,0.664997,0.647850,0.632327,0.673430,0.674243,0.354770,0.375139,0.392136,0.259079,0.235894
4,0.521616,0.481960,0.497882,0.424561,0.471703,0.461276,0.440147,0.176911,0.427434,0.416903,...,0.563411,0.547835,0.563921,0.568434,0.576711,0.320389,0.333492,0.341322,0.263476,0.255436


In [12]:
scaled_data

,0,2,4,6,8,10,12,14,16,18,...,917,919,921,923,925,927,929,931,933,935
0,0.427961,0.435677,0.436740,0.415821,0.438565,0.444987,0.464860,0.172802,0.476210,0.480282,...,0.654941,0.644471,0.646571,0.660862,0.661105,0.365628,0.379911,0.393991,0.340217,0.326572
1,0.086924,0.015460,0.108961,0.070948,0.010965,0.043659,0.154643,0.154944,0.184086,0.177992,...,0.572409,0.560926,0.598992,0.573828,0.586091,0.320394,0.333341,0.341992,0.301962,0.288825
2,0.590029,0.608007,0.592665,0.557452,0.608684,0.603127,0.582902,0.244280,0.574486,0.573647,...,0.576204,0.563124,0.566428,0.581813,0.585688,0.311203,0.324836,0.335835,0.289807,0.275156
3,0.448939,0.371288,0.416430,0.314643,0.353757,0.343296,0.330349,0.127264,0.299431,0.277762,...,0.664997,0.647850,0.632327,0.673430,0.674243,0.354770,0.375139,0.392136,0.259079,0.235894
4,0.521616,0.481960,0.497882,0.424561,0.471703,0.461276,0.440147,0.176911,0.427434,0.416903,...,0.563411,0.547835,0.563921,0.568434,0.576711,0.320389,0.333492,0.341322,0.263476,0.255436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1845,0.656825,0.697335,0.664838,0.648534,0.703209,0.697531,0.670576,0.297854,0.664839,0.669385,...,0.632895,0.618856,0.631514,0.638551,0.643549,0.356550,0.369516,0.380486,0.319623,0.308835
1846,0.642771,0.692170,0.655165,0.646625,0.699622,0.695569,0.672451,0.306477,0.672098,0.678833,...,0.634696,0.620988,0.628315,0.640728,0.644033,0.349504,0.363634,0.375904,0.323248,0.305578
1847,0.436422,0.481936,0.459335,0.489502,0.497173,0.513230,0.553044,0.264314,0.590499,0.611166,...,0.581328,0.575411,0.605343,0.581661,0.588524,0.376751,0.379769,0.382978,0.411063,0.395752
1848,0.469397,0.487816,0.480906,0.466176,0.493529,0.499644,0.513705,0.188875,0.524897,0.531409,...,0.666190,0.655899,0.665810,0.671004,0.673514,0.386605,0.398198,0.409306,0.370828,0.356759


In [11]:
scaled_data.to_csv('preprocessed_data.csv', index=False)
labels.to_csv('labels.csv', index=False)